In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import random
import numpy as np
import tensorflow as tf
import re
import cv2
from scipy.stats import f_oneway
from sklearn.preprocessing import LabelEncoder

from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, concatenate
from sklearn.model_selection import train_test_split
import os
import pandas as pd

In [ ]:
import kagglehub

import kagglehub
import os
os.environ["KAGGLE_DATA_DIR"] = "/data/orlando/home/mortaza/.cache/kagglehub"
# Download latest version
full_data = kagglehub.dataset_download("awsaf49/cbis-ddsm-breast-cancer-image-dataset")

print("Path to files:", full_data)




import shutil
import os

# Define source paths
data = "/root/.cache/kagglehub/datasets/awsaf49/cbis-ddsm-breast-cancer-image-dataset"

# Define destination paths
dest_path = "drive/MyDrive/breast_cancer_detection/full_data"

# Ensure destination directories exist
os.makedirs(dest_path, exist_ok=True)

# Move the directories to /content/
shutil.move(data, dest_path)

print("Moved to:", dest_path)


Resuming download from 1219493888 bytes (4099503200 bytes left)...
Resuming download from https://www.kaggle.com/api/v1/datasets/download/awsaf49/cbis-ddsm-breast-cancer-image-dataset?dataset_version_number=1 (1219493888/5318997088) bytes left.


100%|██████████| 4.95G/4.95G [02:53<00:00, 23.6MB/s]

Extracting files...


Path to files: /root/.cache/kagglehub/datasets/awsaf49/cbis-ddsm-breast-cancer-image-dataset/versions/1
Moved to: drive/MyDrive/breast_cancer_detection/full_data


In [ ]:
import pandas as pd
calc_train_df=pd.read_csv('/content/drive/MyDrive/breast_cancer_detection/full_data/cbis-ddsm-breast-cancer-image-dataset/versions/1/csv/calc_case_description_train_set.csv')
calc_test_df=pd.read_csv('/content/drive/MyDrive/breast_cancer_detection/full_data/cbis-ddsm-breast-cancer-image-dataset/versions/1/csv/calc_case_description_test_set.csv')
mass_train_df=pd.read_csv('/content/drive/MyDrive/breast_cancer_detection/full_data/cbis-ddsm-breast-cancer-image-dataset/versions/1/csv/mass_case_description_train_set.csv')
mass_test_df=pd.read_csv('/content/drive/MyDrive/breast_cancer_detection/full_data/cbis-ddsm-breast-cancer-image-dataset/versions/1/csv/mass_case_description_test_set.csv')


In [ ]:
calc_train_df.shape,mass_train_df.shape

((1546, 14), (1318, 14))

In [ ]:
calc_test_df.shape,mass_test_df.shape

((326, 14), (378, 14))

In [ ]:
calc_train_df.head()

,patient_id,breast density,left or right breast,image view,abnormality id,abnormality type,calc type,calc distribution,assessment,pathology,subtlety,image file path,cropped image file path,ROI mask file path
0,P_00005,3,RIGHT,CC,1,calcification,AMORPHOUS,CLUSTERED,3,MALIGNANT,3,Calc-Training_P_00005_RIGHT_CC/1.3.6.1.4.1.959...,Calc-Training_P_00005_RIGHT_CC_1/1.3.6.1.4.1.9...,Calc-Training_P_00005_RIGHT_CC_1/1.3.6.1.4.1.9...
1,P_00005,3,RIGHT,MLO,1,calcification,AMORPHOUS,CLUSTERED,3,MALIGNANT,3,Calc-Training_P_00005_RIGHT_MLO/1.3.6.1.4.1.95...,Calc-Training_P_00005_RIGHT_MLO_1/1.3.6.1.4.1....,Calc-Training_P_00005_RIGHT_MLO_1/1.3.6.1.4.1....
2,P_00007,4,LEFT,CC,1,calcification,PLEOMORPHIC,LINEAR,4,BENIGN,4,Calc-Training_P_00007_LEFT_CC/1.3.6.1.4.1.9590...,Calc-Training_P_00007_LEFT_CC_1/1.3.6.1.4.1.95...,Calc-Training_P_00007_LEFT_CC_1/1.3.6.1.4.1.95...
3,P_00007,4,LEFT,MLO,1,calcification,PLEOMORPHIC,LINEAR,4,BENIGN,4,Calc-Training_P_00007_LEFT_MLO/1.3.6.1.4.1.959...,Calc-Training_P_00007_LEFT_MLO_1/1.3.6.1.4.1.9...,Calc-Training_P_00007_LEFT_MLO_1/1.3.6.1.4.1.9...
4,P_00008,1,LEFT,CC,1,calcification,NaN,REGIONAL,2,BENIGN_WITHOUT_CALLBACK,3,Calc-Training_P_00008_LEFT_CC/1.3.6.1.4.1.9590...,Calc-Training_P_00008_LEFT_CC_1/1.3.6.1.4.1.95...,Calc-Training_P_00008_LEFT_CC_1/1.3.6.1.4.1.95...


In [ ]:
mass_train_df.head()

,patient_id,breast_density,left or right breast,image view,abnormality id,abnormality type,mass shape,mass margins,assessment,pathology,subtlety,image file path,cropped image file path,ROI mask file path
0,P_00001,3,LEFT,CC,1,mass,IRREGULAR-ARCHITECTURAL_DISTORTION,SPICULATED,4,MALIGNANT,4,Mass-Training_P_00001_LEFT_CC/1.3.6.1.4.1.9590...,Mass-Training_P_00001_LEFT_CC_1/1.3.6.1.4.1.95...,Mass-Training_P_00001_LEFT_CC_1/1.3.6.1.4.1.95...
1,P_00001,3,LEFT,MLO,1,mass,IRREGULAR-ARCHITECTURAL_DISTORTION,SPICULATED,4,MALIGNANT,4,Mass-Training_P_00001_LEFT_MLO/1.3.6.1.4.1.959...,Mass-Training_P_00001_LEFT_MLO_1/1.3.6.1.4.1.9...,Mass-Training_P_00001_LEFT_MLO_1/1.3.6.1.4.1.9...
2,P_00004,3,LEFT,CC,1,mass,ARCHITECTURAL_DISTORTION,ILL_DEFINED,4,BENIGN,3,Mass-Training_P_00004_LEFT_CC/1.3.6.1.4.1.9590...,Mass-Training_P_00004_LEFT_CC_1/1.3.6.1.4.1.95...,Mass-Training_P_00004_LEFT_CC_1/1.3.6.1.4.1.95...
3,P_00004,3,LEFT,MLO,1,mass,ARCHITECTURAL_DISTORTION,ILL_DEFINED,4,BENIGN,3,Mass-Training_P_00004_LEFT_MLO/1.3.6.1.4.1.959...,Mass-Training_P_00004_LEFT_MLO_1/1.3.6.1.4.1.9...,Mass-Training_P_00004_LEFT_MLO_1/1.3.6.1.4.1.9...
4,P_00004,3,RIGHT,MLO,1,mass,OVAL,CIRCUMSCRIBED,4,BENIGN,5,Mass-Training_P_00004_RIGHT_MLO/1.3.6.1.4.1.95...,Mass-Training_P_00004_RIGHT_MLO_1/1.3.6.1.4.1....,Mass-Training_P_00004_RIGHT_MLO_1/1.3.6.1.4.1....


In [ ]:
dicom_data=pd.read_csv('/content/drive/MyDrive/breast_cancer_detection/full_data/cbis-ddsm-breast-cancer-image-dataset/versions/1/csv/dicom_info.csv')

In [ ]:
full_mammogram_images = dicom_data[dicom_data.SeriesDescription == 'full mammogram images'].image_path = dicom_data[dicom_data.SeriesDescription == 'full mammogram images'].image_path
cropped_images = dicom_data[dicom_data.SeriesDescription == 'cropped images'].image_path
roi_images=dicom_data[dicom_data.SeriesDescription == 'ROI mask images'].image_path

<ipython-input-9-fdb86cd20d37>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_mammogram_images = dicom_data[dicom_data.SeriesDescription == 'full mammogram images'].image_path = dicom_data[dicom_data.SeriesDescription == 'full mammogram images'].image_path


In [ ]:
image_dir = '/content/drive/MyDrive/breast_cancer_detection/full_data/cbis-ddsm-breast-cancer-image-dataset/versions/1/jpeg/'
full_mammogram_images = full_mammogram_images.apply(lambda x: x.replace('CBIS-DDSM/jpeg', image_dir))
cropped_images = cropped_images.apply(lambda x: x.replace('CBIS-DDSM/jpeg', image_dir))
roi_images=roi_images.apply(lambda x: x.replace('CBIS-DDSM/jpeg', image_dir))

In [ ]:
dicom_cleaning_data = dicom_data.copy()
dicom_cleaning_data['image_path'] = dicom_cleaning_data['image_path'].str.replace('CBIS-DDSM/jpeg/', image_dir)

In [ ]:
label_mapp = {'BENIGN': 0, 'BENIGN_WITHOUT_CALLBACK': 0, 'MALIGNANT': 1}
calc_train_df['tumor'] = calc_train_df['pathology'].map(label_mapp)
calc_test_df['tumor'] = calc_test_df['pathology'].map(label_mapp)
mass_train_df['tumor'] = mass_train_df['pathology'].map(label_mapp)
mass_test_df['tumor'] = mass_test_df['pathology'].map(label_mapp)

In [ ]:
calc_train_df['tumor'].value_counts(),calc_test_df['tumor'].value_counts(),mass_train_df['tumor'].value_counts(),mass_test_df['tumor'].value_counts()

(tumor
 0    1002
 1     544
 Name: count, dtype: int64,
 tumor
 0    197
 1    129
 Name: count, dtype: int64,
 tumor
 0    681
 1    637
 Name: count, dtype: int64,
 tumor
 0    231
 1    147
 Name: count, dtype: int64)

In [ ]:
calc_train_df.columns

Index(['patient_id', 'breast density', 'left or right breast', 'image view',
       'abnormality id', 'abnormality type', 'calc type', 'calc distribution',
       'assessment', 'pathology', 'subtlety', 'image file path',
       'cropped image file path', 'ROI mask file path', 'tumor'],
      dtype='object')

In [ ]:
dicom_data.head()

,file_path,image_path,AccessionNumber,BitsAllocated,BitsStored,BodyPartExamined,Columns,ContentDate,ContentTime,ConversionType,...,SecondaryCaptureDeviceManufacturerModelName,SeriesDescription,SeriesInstanceUID,SeriesNumber,SmallestImagePixelValue,SpecificCharacterSet,StudyDate,StudyID,StudyInstanceUID,StudyTime
0,CBIS-DDSM/dicom/1.3.6.1.4.1.9590.100.1.2.12930...,CBIS-DDSM/jpeg/1.3.6.1.4.1.9590.100.1.2.129308...,NaN,16,16,BREAST,351,20160426,131732.685,WSD,...,MATLAB,cropped images,1.3.6.1.4.1.9590.100.1.2.129308726812851964007...,1,23078,ISO_IR 100,20160720.0,DDSM,1.3.6.1.4.1.9590.100.1.2.271867287611061855725...,214951.0
1,CBIS-DDSM/dicom/1.3.6.1.4.1.9590.100.1.2.24838...,CBIS-DDSM/jpeg/1.3.6.1.4.1.9590.100.1.2.248386...,NaN,16,16,BREAST,3526,20160426,143829.101,WSD,...,MATLAB,full mammogram images,1.3.6.1.4.1.9590.100.1.2.248386742010678582309...,1,0,ISO_IR 100,20160720.0,DDSM,1.3.6.1.4.1.9590.100.1.2.161516517311681906612...,193426.0
2,CBIS-DDSM/dicom/1.3.6.1.4.1.9590.100.1.2.26721...,CBIS-DDSM/jpeg/1.3.6.1.4.1.9590.100.1.2.267213...,NaN,16,16,BREAST,1546,20160503,111956.298,WSD,...,MATLAB,full mammogram images,1.3.6.1.4.1.9590.100.1.2.267213171011171858918...,1,0,ISO_IR 100,20160807.0,DDSM,1.3.6.1.4.1.9590.100.1.2.291043622711253836701...,161814.0
3,CBIS-DDSM/dicom/1.3.6.1.4.1.9590.100.1.2.38118...,CBIS-DDSM/jpeg/1.3.6.1.4.1.9590.100.1.2.381187...,NaN,16,16,BREAST,97,20160503,115347.770,WSD,...,MATLAB,cropped images,1.3.6.1.4.1.9590.100.1.2.381187369611524586537...,1,32298,ISO_IR 100,20170829.0,DDSM,1.3.6.1.4.1.9590.100.1.2.335006093711888937440...,180109.0
4,CBIS-DDSM/dicom/1.3.6.1.4.1.9590.100.1.2.38118...,CBIS-DDSM/jpeg/1.3.6.1.4.1.9590.100.1.2.381187...,NaN,8,8,Left Breast,3104,20160503,115347.770,WSD,...,MATLAB,NaN,1.3.6.1.4.1.9590.100.1.2.381187369611524586537...,1,0,ISO_IR 100,NaN,DDSM,1.3.6.1.4.1.9590.100.1.2.335006093711888937440...,NaN


In [ ]:
# dicom_model = dicom_data.copy()
# dicom_model['image_path'] = dicom_cleaning_data['image_path'].str.replace('CBIS-DDSM/jpeg/', image_dir)

# #def load_and_process_image(image_path):
#  #   image = load_img(image_path, target_size=(224, 224), color_mode="grayscale")
#   #  image = img_to_array(image) / 255.0
#    # return image



# def match_full(file_path):
#     patientID = file_path.split('/')[0]
#     series_description = 'full mammogram images'
#     filtered_df = dicom_cleaning_data[
#         (dicom_cleaning_data['SeriesDescription'] == series_description) &
#         (dicom_cleaning_data['PatientName'] == patientID)
#     ]
#     if filtered_df.empty:
#         return None
#     return filtered_df['image_path'].iloc[0]

# def match_cropped(file_path):
#     patientID = file_path.split('/')[0]
#     series_description = 'cropped images'
#     filtered_df = dicom_cleaning_data[
#         (dicom_cleaning_data['SeriesDescription'] == series_description) &
#         (dicom_cleaning_data['PatientName'] == patientID)
#     ]
#     if filtered_df.empty:
#         return None
#     return filtered_df['image_path'].iloc[0]

# def load_data(df):
#     full_imgs = []
#     cropped_imgs = []
# #'image view','left or right breast','calc type','calc distribution','breast density','assessment','tumor'
#     tumors = []
#     image_views=[]
#     left_or_right_breasts=[]
#     calc_types=[]
#     calc_distributions=[]
#     breast_densities=[]
#     assessments=[]






#     for _, row in df.iterrows():
#         full_img_path = match_full(row['image file path'])
#         if full_img_path is None:
#             continue

#         cropped_img_path = match_cropped(row['cropped image file path'])
#         if cropped_img_path is None:
#             continue

#         roi_img_path = row['ROI mask file path']
#         if not os.path.exists(roi_img_path):
#             continue
#         if (
#             os.path.exists(full_img_path) and
#             os.path.exists(cropped_img_path) and
#             os.path.exists(roi_img_path)
#           ):
#             full_imgs.append(load_and_process_image(full_img_path))
#             cropped_imgs.append(load_and_process_image(cropped_img_path))
#             tumors.append(row['tumor'])
#             image_views.append(row['image view'])
#             left_or_right_breasts.append(row['left or right breast'])
#             calc_types.append(row['calc type'])
#             calc_distributions.append(row['calc distribution'])
#             breast_densities.append(row['breast_density'])
#             assessments.append(row['assessment'])

#     return np.array(full_imgs), np.array(cropped_imgs), np.array(tumors),np.array(image_views),np.array(left_or_right_breasts),np.array(calc_types),np.array(calc_distributions),np.array(breast_densities),np.array(assessments)

In [ ]:
def match_full(file_path):
    patientID = file_path.split('/')[0]
    series_description = 'full mammogram images'
    filtered_df = dicom_cleaning_data[
        (dicom_cleaning_data['SeriesDescription'] == series_description) &
        (dicom_cleaning_data['PatientName'] == patientID)
    ]
    if filtered_df.empty:
        return None
    return filtered_df['image_path'].iloc[0]

def match_cropped(file_path):
    patientID = file_path.split('/')[0]
    series_description = 'cropped images'
    filtered_df = dicom_cleaning_data[
        (dicom_cleaning_data['SeriesDescription'] == series_description) &
        (dicom_cleaning_data['PatientName'] == patientID)
    ]
    if filtered_df.empty:
        return None
    return filtered_df['image_path'].iloc[0]

def match_roi(file_path):
    patientID = file_path.split('/')[0]
    series_description = 'ROI mask images'
    filtered_df = dicom_cleaning_data[
        (dicom_cleaning_data['SeriesDescription'] == series_description) &
        (dicom_cleaning_data['PatientName'] == patientID)
    ]
    if filtered_df.empty:
        return None
    return filtered_df['image_path'].iloc[0]


In [ ]:
import pandas as pd
from tqdm import tqdm
def load_data(df):
    # Create an empty list to store rows
    data_rows = []

    for _, row in tqdm(df.iterrows()):
        full_img_path = match_full(row['image file path'])
        if full_img_path is None:
            continue

        cropped_img_path = match_cropped(row['cropped image file path'])
        if cropped_img_path is None:
            continue

        roi_img_path = match_roi(row['ROI mask file path'])
        if roi_img_path is None:
            continue

        if os.path.exists(full_img_path) and os.path.exists(cropped_img_path) and os.path.exists(roi_img_path):
            # Create a row with the required data
            data_row = {
                'full_img_path': full_img_path,
                'cropped_img_path': cropped_img_path,
                'roi_img_path': roi_img_path,
                'tumor': row['tumor'],
                'image_view': row['image view'],
                'left_or_right_breast': row['left or right breast'],
                'calc_type': row['calc type'],
                'calc_distribution': row['calc distribution'],
                'breast_density': row['breast_density'],
                'assessment': row['assessment']
            }
            data_rows.append(data_row)

    # Convert the list of rows into a DataFrame
    data_df = pd.DataFrame(data_rows)

    return data_df


In [ ]:
calc_train_model = calc_train_df.copy()
calc_train_model = calc_train_model.drop_duplicates(subset=['image file path']).reset_index(drop=True)
mass_train_model = mass_train_df.copy()
mass_train_model = mass_train_model.drop_duplicates(subset=['image file path']).reset_index(drop=True)

In [ ]:
calc_test_model = calc_test_df.copy()
calc_test_model = calc_test_model.drop_duplicates(subset=['image file path']).reset_index(drop=True)
mass_test_model = mass_test_df.copy()
mass_test_model = mass_test_model.drop_duplicates(subset=['image file path']).reset_index(drop=True)

In [ ]:
dicom_cleaning_data.columns

Index(['file_path', 'image_path', 'AccessionNumber', 'BitsAllocated',
       'BitsStored', 'BodyPartExamined', 'Columns', 'ContentDate',
       'ContentTime', 'ConversionType', 'HighBit', 'InstanceNumber',
       'LargestImagePixelValue', 'Laterality', 'Modality', 'PatientBirthDate',
       'PatientID', 'PatientName', 'PatientOrientation', 'PatientSex',
       'PhotometricInterpretation', 'PixelRepresentation',
       'ReferringPhysicianName', 'Rows', 'SOPClassUID', 'SOPInstanceUID',
       'SamplesPerPixel', 'SecondaryCaptureDeviceManufacturer',
       'SecondaryCaptureDeviceManufacturerModelName', 'SeriesDescription',
       'SeriesInstanceUID', 'SeriesNumber', 'SmallestImagePixelValue',
       'SpecificCharacterSet', 'StudyDate', 'StudyID', 'StudyInstanceUID',
       'StudyTime'],
      dtype='object')

In [ ]:
calc_train_model.head()

,patient_id,breast density,left or right breast,image view,abnormality id,abnormality type,calc type,calc distribution,assessment,pathology,subtlety,image file path,cropped image file path,ROI mask file path,tumor
0,P_00005,3,RIGHT,CC,1,calcification,AMORPHOUS,CLUSTERED,3,MALIGNANT,3,Calc-Training_P_00005_RIGHT_CC/1.3.6.1.4.1.959...,Calc-Training_P_00005_RIGHT_CC_1/1.3.6.1.4.1.9...,Calc-Training_P_00005_RIGHT_CC_1/1.3.6.1.4.1.9...,1
1,P_00005,3,RIGHT,MLO,1,calcification,AMORPHOUS,CLUSTERED,3,MALIGNANT,3,Calc-Training_P_00005_RIGHT_MLO/1.3.6.1.4.1.95...,Calc-Training_P_00005_RIGHT_MLO_1/1.3.6.1.4.1....,Calc-Training_P_00005_RIGHT_MLO_1/1.3.6.1.4.1....,1
2,P_00007,4,LEFT,CC,1,calcification,PLEOMORPHIC,LINEAR,4,BENIGN,4,Calc-Training_P_00007_LEFT_CC/1.3.6.1.4.1.9590...,Calc-Training_P_00007_LEFT_CC_1/1.3.6.1.4.1.95...,Calc-Training_P_00007_LEFT_CC_1/1.3.6.1.4.1.95...,0
3,P_00007,4,LEFT,MLO,1,calcification,PLEOMORPHIC,LINEAR,4,BENIGN,4,Calc-Training_P_00007_LEFT_MLO/1.3.6.1.4.1.959...,Calc-Training_P_00007_LEFT_MLO_1/1.3.6.1.4.1.9...,Calc-Training_P_00007_LEFT_MLO_1/1.3.6.1.4.1.9...,0
4,P_00008,1,LEFT,CC,1,calcification,NaN,REGIONAL,2,BENIGN_WITHOUT_CALLBACK,3,Calc-Training_P_00008_LEFT_CC/1.3.6.1.4.1.9590...,Calc-Training_P_00008_LEFT_CC_1/1.3.6.1.4.1.95...,Calc-Training_P_00008_LEFT_CC_1/1.3.6.1.4.1.95...,0


In [ ]:
calc_train_model.columns,mass_train_model.columns

(Index(['patient_id', 'breast density', 'left or right breast', 'image view',
        'abnormality id', 'abnormality type', 'calc type', 'calc distribution',
        'assessment', 'pathology', 'subtlety', 'image file path',
        'cropped image file path', 'ROI mask file path', 'tumor'],
       dtype='object'),
 Index(['patient_id', 'breast_density', 'left or right breast', 'image view',
        'abnormality id', 'abnormality type', 'mass shape', 'mass margins',
        'assessment', 'pathology', 'subtlety', 'image file path',
        'cropped image file path', 'ROI mask file path', 'tumor'],
       dtype='object'))

In [ ]:
calc_train_model.columns==mass_train_model.columns

array([ True, False,  True,  True,  True,  True, False, False,  True,
        True,  True,  True,  True,  True,  True])

In [ ]:
calc_train_model.rename(columns={'breast density':'breast_density'},inplace=True)
calc_test_model.rename(columns={'breast density':'breast_density'},inplace=True)

In [ ]:
calc_train_model['mass shape'] = np.nan
calc_train_model['mass margins'] = np.nan

calc_test_model['mass shape'] = np.nan
calc_test_model['mass margins'] = np.nan

In [ ]:
mass_train_model['calc type'] = np.nan
mass_train_model['calc distribution'] = np.nan

mass_test_model['calc type'] = np.nan
mass_test_model['calc distribution'] = np.nan

In [ ]:
common_columns = ['patient_id', 'breast_density', 'left or right breast', 'image view',
        'abnormality id', 'abnormality type', 'calc type', 'calc distribution',
        'assessment', 'pathology', 'subtlety', 'image file path',
        'cropped image file path', 'ROI mask file path', 'tumor', 'mass shape',
        'mass margins']

calc_train_model = calc_train_model[common_columns]
calc_test_model = calc_test_model[common_columns]
mass_train_model = mass_train_model[common_columns]
mass_test_model = mass_test_model[common_columns]


In [ ]:
calc_train_model.head()

,patient_id,breast_density,left or right breast,image view,abnormality id,abnormality type,calc type,calc distribution,assessment,pathology,subtlety,image file path,cropped image file path,ROI mask file path,tumor,mass shape,mass margins
0,P_00005,3,RIGHT,CC,1,calcification,AMORPHOUS,CLUSTERED,3,MALIGNANT,3,Calc-Training_P_00005_RIGHT_CC/1.3.6.1.4.1.959...,Calc-Training_P_00005_RIGHT_CC_1/1.3.6.1.4.1.9...,Calc-Training_P_00005_RIGHT_CC_1/1.3.6.1.4.1.9...,1,NaN,NaN
1,P_00005,3,RIGHT,MLO,1,calcification,AMORPHOUS,CLUSTERED,3,MALIGNANT,3,Calc-Training_P_00005_RIGHT_MLO/1.3.6.1.4.1.95...,Calc-Training_P_00005_RIGHT_MLO_1/1.3.6.1.4.1....,Calc-Training_P_00005_RIGHT_MLO_1/1.3.6.1.4.1....,1,NaN,NaN
2,P_00007,4,LEFT,CC,1,calcification,PLEOMORPHIC,LINEAR,4,BENIGN,4,Calc-Training_P_00007_LEFT_CC/1.3.6.1.4.1.9590...,Calc-Training_P_00007_LEFT_CC_1/1.3.6.1.4.1.95...,Calc-Training_P_00007_LEFT_CC_1/1.3.6.1.4.1.95...,0,NaN,NaN
3,P_00007,4,LEFT,MLO,1,calcification,PLEOMORPHIC,LINEAR,4,BENIGN,4,Calc-Training_P_00007_LEFT_MLO/1.3.6.1.4.1.959...,Calc-Training_P_00007_LEFT_MLO_1/1.3.6.1.4.1.9...,Calc-Training_P_00007_LEFT_MLO_1/1.3.6.1.4.1.9...,0,NaN,NaN
4,P_00008,1,LEFT,CC,1,calcification,NaN,REGIONAL,2,BENIGN_WITHOUT_CALLBACK,3,Calc-Training_P_00008_LEFT_CC/1.3.6.1.4.1.9590...,Calc-Training_P_00008_LEFT_CC_1/1.3.6.1.4.1.95...,Calc-Training_P_00008_LEFT_CC_1/1.3.6.1.4.1.95...,0,NaN,NaN


In [ ]:
train_dataset = pd.concat([calc_train_model, mass_train_model], ignore_index=True)
test_dataset = pd.concat([calc_test_model, mass_test_model], ignore_index=True)

In [ ]:
final_train_df=load_data(train_dataset)

Streaming output truncated to the last 5000 lines.
/content/drive/MyDrive/breast_cancer_detection/full_data/cbis-ddsm-breast-cancer-image-dataset/versions/1/jpeg/1.3.6.1.4.1.9590.100.1.2.247478563813385502806126434882707296853/1-123.jpg
/content/drive/MyDrive/breast_cancer_detection/full_data/cbis-ddsm-breast-cancer-image-dataset/versions/1/jpeg/1.3.6.1.4.1.9590.100.1.2.285192361611580457019084240851077706099/1-088.jpg
/content/drive/MyDrive/breast_cancer_detection/full_data/cbis-ddsm-breast-cancer-image-dataset/versions/1/jpeg/1.3.6.1.4.1.9590.100.1.2.285192361611580457019084240851077706099/2-064.jpg
.>......... ALL EXISTS 
/content/drive/MyDrive/breast_cancer_detection/full_data/cbis-ddsm-breast-cancer-image-dataset/versions/1/jpeg/1.3.6.1.4.1.9590.100.1.2.253455683911605267101949183330847157668/1-124.jpg
/content/drive/MyDrive/breast_cancer_detection/full_data/cbis-ddsm-breast-cancer-image-dataset/versions/1/jpeg/1.3.6.1.4.1.9590.100.1.2.411726412611642933330035375703614362414/1-087

In [ ]:
final_test_df=load_data(test_dataset)

70it [00:00, 311.03it/s]

No matching full mammogram images for Calc-Test_P_00038_LEFT_CC
No matching full mammogram images for Calc-Test_P_00038_LEFT_MLO
No matching full mammogram images for Calc-Test_P_00038_RIGHT_CC
No matching full mammogram images for Calc-Test_P_00038_RIGHT_MLO
No matching full mammogram images for Calc-Test_P_00041_LEFT_CC
No matching full mammogram images for Calc-Test_P_00041_LEFT_MLO
No matching full mammogram images for Calc-Test_P_00077_LEFT_CC
No matching full mammogram images for Calc-Test_P_00077_LEFT_MLO
No matching full mammogram images for Calc-Test_P_00077_RIGHT_CC
No matching full mammogram images for Calc-Test_P_00077_RIGHT_MLO
No matching full mammogram images for Calc-Test_P_00100_RIGHT_CC
No matching full mammogram images for Calc-Test_P_00100_RIGHT_MLO
No matching full mammogram images for Calc-Test_P_00127_RIGHT_CC
No matching full mammogram images for Calc-Test_P_00127_RIGHT_MLO
No matching full mammogram images for Calc-Test_P_00132_LEFT_MLO
No matching full mammogr

102it [00:00, 285.32it/s]

No matching full mammogram images for Calc-Test_P_00497_LEFT_CC
No matching full mammogram images for Calc-Test_P_00497_LEFT_MLO
No matching full mammogram images for Calc-Test_P_00537_RIGHT_MLO
No matching full mammogram images for Calc-Test_P_00562_LEFT_CC
No matching full mammogram images for Calc-Test_P_00562_LEFT_MLO
No matching ROI mask images for Calc-Test_P_00562_RIGHT_CC_1
No matching ROI mask images for Calc-Test_P_00562_RIGHT_MLO_1
No matching full mammogram images for Calc-Test_P_00570_LEFT_MLO
No matching full mammogram images for Calc-Test_P_00579_LEFT_MLO
No matching full mammogram images for Calc-Test_P_00589_LEFT_CC
No matching full mammogram images for Calc-Test_P_00589_LEFT_MLO
No matching ROI mask images for Calc-Test_P_00608_LEFT_CC_1
No matching full mammogram images for Calc-Test_P_00608_LEFT_MLO
No matching full mammogram images for Calc-Test_P_00620_LEFT_CC
No matching full mammogram images for Calc-Test_P_00643_LEFT_CC
No matching full mammogram images for Cal

161it [00:00, 286.41it/s]

...

227it [00:00, 300.48it/s]

No matching full mammogram images for Calc-Test_P_01465_LEFT_MLO
No matching full mammogram images for Calc-Test_P_01471_RIGHT_CC
No matching full mammogram images for Calc-Test_P_01471_RIGHT_MLO
No matching full mammogram images for Calc-Test_P_01476_RIGHT_CC
No matching full mammogram images for Calc-Test_P_01476_RIGHT_MLO
No matching ROI mask images for Calc-Test_P_01483_LEFT_CC_1
No matching full mammogram images for Calc-Test_P_01483_LEFT_MLO
No matching full mammogram images for Calc-Test_P_01490_LEFT_CC
No matching full mammogram images for Calc-Test_P_01490_LEFT_MLO
No matching full mammogram images for Calc-Test_P_01502_LEFT_CC
No matching full mammogram images for Calc-Test_P_01502_LEFT_MLO
No matching full mammogram images for Calc-Test_P_01523_LEFT_MLO
No matching full mammogram images for Calc-Test_P_01534_LEFT_CC
No matching full mammogram images for Calc-Test_P_01534_LEFT_MLO
No matching full mammogram images for Calc-Test_P_01535_LEFT_CC
No matching full mammogram image

287it [00:01, 269.55it/s]

No matching ROI mask images for Calc-Test_P_01882_LEFT_CC_1
No matching ROI mask images for Calc-Test_P_01882_LEFT_MLO_1
No matching full mammogram images for Calc-Test_P_01883_RIGHT_CC
No matching full mammogram images for Calc-Test_P_01883_RIGHT_MLO
No matching full mammogram images for Calc-Test_P_02102_LEFT_CC
No matching full mammogram images for Calc-Test_P_02102_LEFT_MLO
No matching full mammogram images for Calc-Test_P_02139_LEFT_CC
No matching full mammogram images for Calc-Test_P_02139_LEFT_MLO
No matching full mammogram images for Calc-Test_P_02153_RIGHT_CC
No matching full mammogram images for Calc-Test_P_02153_RIGHT_MLO
No matching full mammogram images for Calc-Test_P_02159_LEFT_CC
No matching full mammogram images for Calc-Test_P_02159_LEFT_MLO
No matching full mammogram images for Calc-Test_P_02176_RIGHT_CC
No matching full mammogram images for Calc-Test_P_02176_RIGHT_MLO
No matching full mammogram images for Calc-Test_P_02198_LEFT_CC
No matching full mammogram images f

645it [00:04, 160.67it/s]


In [ ]:
final_train_df.shape,final_test_df.shape

((2458, 10), (361, 10))

In [ ]:
final_train_df.to_csv('/content/drive/MyDrive/breast_cancer_detection/final_train_df.csv',index=False)
final_test_df.to_csv('/content/drive/MyDrive/breast_cancer_detection/final_test_df.csv',index=False)

In [ ]:
final_train_df.head()

,full_img_path,cropped_img_path,roi_img_path,tumor,image_view,left_or_right_breast,calc_type,calc_distribution,breast_density,assessment
0,/content/drive/MyDrive/breast_cancer_detection...,/content/drive/MyDrive/breast_cancer_detection...,/content/drive/MyDrive/breast_cancer_detection...,1,0,1,0,0,3,3
1,/content/drive/MyDrive/breast_cancer_detection...,/content/drive/MyDrive/breast_cancer_detection...,/content/drive/MyDrive/breast_cancer_detection...,1,1,1,0,0,3,3
2,/content/drive/MyDrive/breast_cancer_detection...,/content/drive/MyDrive/breast_cancer_detection...,/content/drive/MyDrive/breast_cancer_detection...,0,0,0,18,4,4,4
3,/content/drive/MyDrive/breast_cancer_detection...,/content/drive/MyDrive/breast_cancer_detection...,/content/drive/MyDrive/breast_cancer_detection...,0,1,0,18,4,4,4
4,/content/drive/MyDrive/breast_cancer_detection...,/content/drive/MyDrive/breast_cancer_detection...,/content/drive/MyDrive/breast_cancer_detection...,0,0,0,45,6,1,2


In [ ]:
final_train_df=pd.read_csv('/content/drive/MyDrive/breast_cancer_detection/final_train_df.csv')
final_test_df=pd.read_csv('/content/drive/MyDrive/breast_cancer_detection/final_test_df.csv')

In [ ]:
columns_to_encode=['image_view','left_or_right_breast','calc_type','calc_distribution']

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()
encoders={}

In [ ]:
for column in columns_to_encode:
    encoder = LabelEncoder()
    final_train_df[column] = encoder.fit_transform(final_train_df[column])
    encoders[column] = {
        'encoder': encoder,
        'mapping': dict(zip(encoder.classes_, encoder.transform(encoder.classes_)))
    }

In [ ]:
for column in columns_to_encode:
    test_encoder = encoders[column]['encoder']  # Get the encoder used on training data

    # Handle unseen labels by assigning them to a new integer (this avoids KeyError)
    # Using `transform` directly may cause an issue if unseen labels exist in the test data.
    try:
        final_test_df[column] = test_encoder.transform(final_test_df[column])
    except ValueError as e:
        # Handle the case where unseen labels are found
        print(f"Warning: Unseen labels found in column {column}. Mapping unseen labels to a default value.")
        final_test_df[column] = final_test_df[column].apply(lambda x: test_encoder.transform([x])[0] if x in test_encoder.classes_ else -1)

    encoders[column] = {
        'encoder': test_encoder,
        'mapping': dict(zip(test_encoder.classes_, test_encoder.transform(test_encoder.classes_)))
    }

In [ ]:
encoders

{'image_view': {'encoder': LabelEncoder(), 'mapping': {'CC': 0, 'MLO': 1}},
 'left_or_right_breast': {'encoder': LabelEncoder(),
  'mapping': {'LEFT': 0, 'RIGHT': 1}},
 'calc_type': {'encoder': LabelEncoder(),
  'mapping': {'AMORPHOUS': 0,
   'AMORPHOUS-PLEOMORPHIC': 1,
   'AMORPHOUS-ROUND_AND_REGULAR': 2,
   'COARSE': 3,
   'COARSE-LUCENT_CENTER': 4,
   'COARSE-PLEOMORPHIC': 5,
   'COARSE-ROUND_AND_REGULAR': 6,
   'COARSE-ROUND_AND_REGULAR-LUCENT_CENTER': 7,
   'COARSE-ROUND_AND_REGULAR-LUCENT_CENTERED': 8,
   'DYSTROPHIC': 9,
   'EGGSHELL': 10,
   'FINE_LINEAR_BRANCHING': 11,
   'LARGE_RODLIKE': 12,
   'LARGE_RODLIKE-ROUND_AND_REGULAR': 13,
   'LUCENT_CENTER': 14,
   'LUCENT_CENTER-PUNCTATE': 15,
   'LUCENT_CENTERED': 16,
   'MILK_OF_CALCIUM': 17,
   'PLEOMORPHIC': 18,
   'PLEOMORPHIC-FINE_LINEAR_BRANCHING': 19,
   'PLEOMORPHIC-PLEOMORPHIC': 20,
   'PUNCTATE': 21,
   'PUNCTATE-AMORPHOUS': 22,
   'PUNCTATE-FINE_LINEAR_BRANCHING': 23,
   'PUNCTATE-LUCENT_CENTER': 24,
   'PUNCTATE-PLEOM

In [ ]:
edf=pd.DataFrame(encoders)

In [ ]:
edf.to_csv('/content/drive/MyDrive/breast_cancer_detection/encoders.csv',index=False)

In [ ]:
final_train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2458 entries, 0 to 2457
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   full_img_path         2458 non-null   object
 1   cropped_img_path      2458 non-null   object
 2   roi_img_path          2458 non-null   object
 3   tumor                 2458 non-null   int64 
 4   image_view            2458 non-null   int64 
 5   left_or_right_breast  2458 non-null   int64 
 6   calc_type             2458 non-null   int64 
 7   calc_distribution     2458 non-null   int64 
 8   breast_density        2458 non-null   int64 
 9   assessment            2458 non-null   int64 
dtypes: int64(7), object(3)
memory usage: 192.2+ KB
